In [4]:
import pandas as pd
import numpy as np
import talib as talib
from pyecharts import options as opts
from pyecharts.charts import *
# from pyecharts.

from datetime import datetime,timedelta
import matplotlib.pyplot as plt
# %matplotlib inline
#正常显示画图时出现的中文和负号
from pylab import mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False

#使用tushare获取交易数据
#设置token
import tushare as ts
#注意token更换为你在tushare网站上获取的
token='36d50a0386e9066b5c79c2de58ea2d3a9b980020041adc910f76bfc7'
pro=ts.pro_api(token)
index={'上证综指': '000001.SH',
        '深证成指': '399001.SZ',
        '沪深300': '000300.SH',
        '创业板指': '399006.SZ',
        '上证50': '000016.SH',
        '中证500': '000905.SH',
        '中小板指': '399005.SZ',
        '上证180': '000010.SH'}
#获取当前交易的股票代码和名称
def get_code():
    df = pro.stock_basic(exchange='', list_status='L')
    codes=df.ts_code.values
    names=df.name.values
    stock=dict(zip(names,codes))
    #合并指数和个股成一个字典
    stocks=dict(stock,**index)
    return stocks

#获取行情数据
def get_daily_data(stock,start,end):
    #如果代码在字典index里，则取的是指数数据
    code=get_code()[stock]
    if code in index.values():
        df=pro.index_daily(ts_code=code,start_date=start, end_date=end)
    #否则取的是个股数据
    else:
        df=pro.daily(ts_code=code, adj='qfq',start_date=start, end_date=end)
    #将交易日期设置为索引值
    df.index=pd.to_datetime(df.trade_date)
    df=df.sort_index()
    #计算收益率
    df['ret']=df.close/df.close.shift(1)-1
    return df

def get_k_line(xdata, kvalus):
    c=(
        Kline()
        .add_xaxis(xdata)
        .add_yaxis(
            'kline',
            kvalus,
            markline_opts=opts.MarkLineOpts(
                data=[
                    opts.MarkLineItem(type_="max", value_dim="close"),
                    opts.MarkLineItem(type_="min", value_dim="close"),
                     ]
            ),
        )
        .set_global_opts(
            yaxis_opts=opts.AxisOpts(is_scale=True),
            xaxis_opts=opts.AxisOpts(
                    is_scale=True,
                    splitarea_opts=opts.SplitAreaOpts(
                        is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                    ),                        
            ),
            datazoom_opts=[opts.DataZoomOpts()],
            title_opts=opts.TitleOpts(title="Kline-DataZoom-slider"),
        )
    )
    return c

def get_line(xdatas, data_dic):
    line= Line()
    line.add_xaxis(xaxis_data=xdatas)
    for key in data_dic:
        value = data_dic[key];
        line.add_yaxis(
            series_name=value['name'],
            y_axis=value['datas'],
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=2, opacity=0.5),
            label_opts=opts.LabelOpts(is_show=False),
        )
    line.set_global_opts(xaxis_opts=opts.AxisOpts(type_="category"))
    return line

sma_periods = [8,17,25,99]
hs=get_daily_data('平安银行','20190101','')[['trade_date','close','open','high','low','vol']]
v1=np.array(hs.loc[:,['open','close','low','high']])
# v= np.array(hs.close)
trade_dates = np.array(hs.trade_date)

sma_dic= {}
for period in sma_periods:
    name = "SMA" + str(period)
    sma = talib.SMA(np.array(hs.close),period)
    sma_dic[name]={}
    sma_dic[name]['name'] = name;
    sma_dic[name]['datas'] = sma.tolist();

    print(trade_dates.tolist())
kline=get_k_line(trade_dates.tolist(),v1.tolist())
kline.render_notebook();
line=get_line(trade_dates.tolist(),sma_dic)
line.render_notebook();

['20190102', '20190103', '20190104', '20190107', '20190108', '20190109', '20190110', '20190111', '20190114', '20190115', '20190116', '20190117', '20190118', '20190121', '20190122', '20190123', '20190124', '20190125', '20190128', '20190129', '20190130', '20190131', '20190201', '20190211', '20190212', '20190213', '20190214', '20190215', '20190218', '20190219', '20190220', '20190221', '20190222', '20190225', '20190226', '20190227', '20190228', '20190301', '20190304', '20190305', '20190306', '20190307', '20190308', '20190311', '20190312', '20190313', '20190314', '20190315', '20190318', '20190319', '20190320', '20190321', '20190322', '20190325', '20190326', '20190327', '20190328', '20190329', '20190401', '20190402', '20190403', '20190404', '20190408', '20190409', '20190410', '20190411', '20190412', '20190415', '20190416', '20190417', '20190418', '20190419', '20190422', '20190423', '20190424', '20190425', '20190426', '20190429', '20190430', '20190506', '20190507', '20190508', '20190509', '20